# Data Wrangling

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 09:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/20 09:59:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Data comes from calls to San Antonio's 311 call center and can be found in the google classroom.

## Reading Data

In [27]:
df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

Like most things spark, there's multiple ways to do it:

In [28]:
# Another method to read csv file
(
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("source.csv")
)

DataFrame[source_id: string, source_username: string]

### Data Schemas

Common Spark Data Types:

- `StringType`
- `IntegerType`
- `FloatType`

StuctType and StructField wrap the above.

- `StructType`
    - `StructField`: `StringType`
    - `StructField`: `IntegerType`

In [29]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)

spark.read.csv("source.csv", header=True, schema=schema)

DataFrame[source_id: string, source_username: string]

### Writing Data

In [30]:
# for demo purposes
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))

mpg.write.json("mpg_json", mode="overwrite")

# Or...
(
    mpg.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("mpg_csv")
)

## Data Preparation

In [19]:
df = spark.read.csv("case.csv", header=True, inferSchema=True)
df.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

### Renaming Columns

In [20]:
df = df.withColumnRenamed("SLA_due_date", "case_due_date")

### Casting Data Types

In [21]:
# demonstrating we only have yes/no in each field
df.groupBy("case_closed", "case_late").count().show()

+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [22]:
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

df.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [24]:
df.groupBy("council_district").count().show()

+----------------+------+
|council_district| count|
+----------------+------+
|               7| 72445|
|               3|102706|
|               8| 42345|
|               0|  3830|
|               5|114609|
|               6| 74095|
|               9| 40916|
|               1|119309|
|              10| 62926|
|               4| 93778|
|               2|114745|
+----------------+------+



In [25]:
# Cast changes to string

df = df.withColumn("council_district", col("council_district").cast("string"))

In [26]:
df.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

In [32]:
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



`to_timestamp` for date conversion

[Java's `SimpleDateFormat`][1] for the format string

[1]: https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html

In [5]:
print("--- Before handling dates")
df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
df.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates


NameError: name 'df' is not defined

### Data Transformations

Normalize the address

In [34]:
print("--- Before")
df.select("request_address").show(5)

df = df.withColumn("request_address", trim(lower(df.request_address)))

print("--- After")
df.select("request_address").show(5)

--- Before


AnalysisException: cannot resolve 'request_address' given input columns: [source_id, source_username];
'Project ['request_address]
+- Relation [source_id#1466,source_username#1467] csv


Number of days to number of weeks

In [4]:
df = df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

df.select("num_days_late", "num_weeks_late").show(5)

NameError: name 'df' is not defined

String formatting

- 1 -> 0001
- 3 -> 0003
- 12 -> 0012

In [3]:
# '%03d' means at least 3 digits, pad with 0sk
#
# In order to use the format_string function the way we are, we'll need to
# convert council_district back to an integer temporarily, but the final output
# will be a string.
df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

df.select("council_district").show(5)

NameError: name 'df' is not defined

In [ ]:
df.printSchema()

In [ ]:
df.show(2, vertical=True, truncate=False)

### New Features

In [2]:
# Zero at end to capture everthing in the regex

df = df.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

df.select("zipcode").show(5)

NameError: name 'df' is not defined

In [ ]:
df.printSchema()

- `case_age`: How old the case is; difference between case opened date and today
- `days_to_closed`: The number of days between when the case was opened and when it was closed
- `case_lifetime`: Number of days between when the case was opened and when it was closed, if the case is still open, the number of days since the case was opened

In [6]:
df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

NameError: name 'df' is not defined

In [ ]:
df.show(2, truncate=False, vertical=True)

### Joining Department Data

In [ ]:
dept = spark.read.csv("dept.csv", header=True, inferSchema=True)
dept.show(100, truncate=False)

Join using `dept_division`.

In [ ]:
(
    df.join(dept, "dept_division", "left")
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(df.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
    .show(2, vertical=True, truncate=False)
)

In [ ]:
df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(df.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

df.show(2, vertical=True)

## Train Test Split

In [7]:
train, test = df.randomSplit([0.8, 0.2], seed=123)

print('train', train.count(), 'x', len(train.columns))
print('test', test.count(), 'x', len(test.columns))

NameError: name 'df' is not defined

In [8]:
train, validate, test = df.randomSplit([0.6, 0.2, 0.2], seed=123)

print('train', train.count(), 'x', len(train.columns))
print('validate', validate.count(), 'x', len(validate.columns))
print('test', test.count(), 'x', len(test.columns))

NameError: name 'df' is not defined